## Suite Brief des Livres

Reprendre votre brief des livres et créer une API FastAPI avec 2 routes simples de votre choix

In [1]:

import sqlite3
from sqlalchemy import create_engine, text
import nest_asyncio
import uvicorn
import pandas as pd
from fastapi import FastAPI, Form, Request
from fastapi.responses import HTMLResponse


In [2]:
df=pd.read_csv('books.csv')
df.head()

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Soumission,One,50.10,In stock,Books
3,Sharp Objects,Four,47.82,In stock,Books
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books


In [3]:
df.shape

(1723, 5)

## Avec MySQL

In [9]:

# Connexion à la base MySQL
engine = create_engine("mysql+mysqlconnector://root@localhost/books")

#

In [10]:
#Insérer des données depuis pandas

df.to_sql('books_manel', con=engine, if_exists='replace', index=False)

-1

In [11]:
#  Pour charger directement un DataFrame
with engine.connect() as conn:
    df_books2 = pd.read_sql(text("SELECT * FROM books_manel"), con=conn)

df_books2.head()

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Soumission,One,50.10,In stock,Books
3,Sharp Objects,Four,47.82,In stock,Books
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books


In [12]:
df_books2.shape

(1723, 5)

## Avec  SQLite

In [13]:
# --- Initialisation de la base SQLite ---
connection = sqlite3.connect("books_db.db")

In [14]:
#Insérer des données depuis pandas

df.to_sql('books_manel', con=connection, if_exists='append', index=False)

1723

In [15]:
# Lecture de la table Books dans un DataFrame
df_books1 = pd.read_sql("SELECT * FROM books_manel;", con=connection)

# Affichage
df_books1.head()

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Sharp Objects,Four,47.82,In stock,Books
3,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books
4,The Requiem Red,One,22.65,In stock,Books


In [16]:
connection.close()

## Mon API

In [17]:

nest_asyncio.apply()
app = FastAPI()

# --- Lire tous les livres ---
@app.get("/livres")
async def get_books():
    conn = sqlite3.connect("books_db.db")
    cur = conn.cursor()
    cur.execute("SELECT * FROM books_manel;")
    rows = cur.fetchall()
    conn.close()
    return {"books": rows}

@app.get("/livres1")
async def get_books():
    conn = sqlite3.connect("books_db.db")

    # Charger en DataFrame
    df_books = pd.read_sql("SELECT * FROM books_manel;", con=conn)
    conn.close()

    # Transformer en JSON type tableau [{col:val, ...}, {...}]
    books_list = df_books.to_dict(orient="records")

    return {"books": books_list}

# --- Ajouter un livre ---
@app.get("/add", response_class=HTMLResponse)
async def add():
    # --- Trois guillemets pour écrire une chaine de caractères avec retour à la ligne ---
    return """
    <h2>Ajouter un livre</h2>
    <form action="/add-book" method="post">
        Title : <input type="text" name="title"><br>
        Rating : <input type="text" name="rating"><br>
        Price : <input type="float" name="price"><br>
        Availability : <input type="text" name="availability"><br>
        Category : <input type="text" name="category"><br>
        <input type="submit" value="Ajouter">
    </form>
    """



# --- Ajouter un livre via POST ---
@app.post("/add-book", response_class=HTMLResponse)
async def add_book(title: str = Form(...), rating: str = Form (...), price: str = Form(...), availability: str = Form(...), category: str = Form(...)):
    try:
        conn = sqlite3.connect("books_db.db")
        cur = conn.cursor()
        cur.execute("INSERT INTO books_manel (title, rating, price, availability, category) VALUES (?, ?, ?, ?, ?)", (title, rating, price, availability, category))
        conn.commit()
        message = f"Le livre \"{title}\" a été ajouté avec succès !"
    except sqlite3.IntegrityError:
        message = f"Erreur : le titre \"{title}\" est déjà utilisé."
    finally:
        conn.close()
    return HTMLResponse(f"<p>{message}</p><a href='/'>Retour</a>")


# --- Supprimer un livre ---
@app.get("/delete-book/{title}")
async def delete_book(title: str):
    conn = sqlite3.connect("books_db.db")
    cur = conn.cursor()
    cur.execute("DELETE FROM books_manel WHERE title = ?", (title,))
    conn.commit()
    conn.close()
    return {"message": f"Le livre \"{title}\" a été supprimé avec succès."}

# --- Lancer l’app ---
uvicorn.run(app, host="127.0.0.1", port=4000)


INFO:     Started server process [14248]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50512 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50512 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50515 - "GET /livres HTTP/1.1" 200 OK
INFO:     127.0.0.1:50518 - "GET /add HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14248]


In [76]:
df

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Soumission,One,50.10,In stock,Books
3,Sharp Objects,Four,47.82,In stock,Books
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books
...,...,...,...,...,...
1718,Mexican Today: New and Rediscovered Recipes fo...,Five,24.91,In stock,Food and Drink
1719,Vegan Vegetarian Omnivore: Dinner for Everyone...,Two,13.66,In stock,Food and Drink
1720,The Smitten Kitchen Cookbook,One,23.59,In stock,Food and Drink
1721,"The Art of Simple Food: Notes, Lessons, and Re...",Three,34.32,In stock,Food and Drink
